# LTI認証連携の設定を行う -- Moodle

---

Moodleとの間で LTI 1.3による LMS(Learning Management System)との認証連携を設定する。

## はじめに

### 概要

CoursewareHub に [LTI 1.3](http://www.imsglobal.org/spec/lti/v1p3/)による LMS(Learning Management System)との認証連携を設定します。LMSを介した認証とLMSから得られた認可情報に基づく CoursewareHub へのアクセス制御を行います。このNotebookでは連携するLMSとしてMoodleを用います。

#### 基本方針

* CoursewareHubの既存の認証機能であるローカルユーザに基づく認証、学認連携に基づく認証と共存して設定可能とする
* LMSで認証されたユーザとJupyterHubユーザとの対応は LMS から得られるメールアドレスを一定ルールで変換することにより行う
    - 変換ルールは学認連携の認証と同じものを用いる
    - LMS のユーザー管理上メールアドレスとユーザーが適切に対応付くように管理されている前提とする
* CoursewareHubでの講師権限、受講者権限の制御は JupyterHub の admin 権限の有無によって制御する
    - LTI 認証連携機能自体は権限に関する制御は行わない

#### システム構成

auth-proxyコンテナのLTI認証連携機能に関する構成を下図に示します。なおローカルユーザ認証、学認連携認証については省略しています。

![lti構成](images/lti-001.png)

### 事前に準備が必要となるものについて

このNotebookを実行するにあたっての前提条件を以下に記します。

* CoursewareHubとLTI認証連携を行う Moodle に管理者としてログインできること
* LTI認証連携を設定するCoursewareHubとMoodleとが互いにアクセス可能であること

### UnitGroup名

CoursewareHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

## Moodleで外部ツールの設定を行う

CoursewareHubとLTI認証連携を行う Moodle にて、外部ツールの設定を行います。

### 事前設定ツールの追加を行う

CoursewareHubを Moodle の事前設定ツールとして登録します。

まずMoodleに管理者としてログインした後に[サイト管理]--[プラグイン]の順にアクセスしてください。表示された画面の「活動モジュール」セクションに下図のような内容が表示されます。

![Moodle-サイト管理画面](images/moodle-411-01.png)

上図の赤丸で示した「ツールを管理する」というリンクを選択すると、下図に示すような外部ツールの管理画面が表示されます。

![Moodle-ツール管理画面1](images/moodle-411-02.png)

上図の赤丸で示した「ツールを手動設定」のリンクを選択してください。下図に示す「外部ツール設定」画面が表示されます。この画面でCoursewareHubと連携するための情報を設定します。

![Moodle-ツール設定画面1](images/moodle-411-03.png)

まず「ツール設定」セクションの入力を行います。上図の青枠で示した箇所への入力が必要となります。入力が必要となる項目名を以下に示します。

* ツール名
* ツールURL
* LTIバージョン
* 公開鍵タイプ
* ログイン開始URL
* リダイレクトURI
* デフォルト起動コンテナ


「ツール名」にはツールを識別するための名前を入力してください。教師がコース内で外部ツールを追加するときに、ここで指定した名前が表示されます。

「ツールURL」には CoursewareHub のURLを入力してください。次のセルを実行すると CoursewareHubのURLが表示されます。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}'

「LTIバージョン」は `LTI 1.3` を選択してください。

「公開鍵タイプ」は`鍵セットURL`を選択してください。

「ログイン開始URL」には CoursewareHubの OpenID Connect/Initialization Endpoint を指定します。次のセルを実行して表示されるURLを入力してください。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}/php/lti/login.php'

「リダイレクトURI」には CoursewareHubの Redirect Endpoint を指定します。次のセルを実行して表示されるURLを入力してください。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}/php/lti/service.php'

「デフォルト起動コンテナ」には「`新しいウィンドウ`」か「`既存のウィンドウ`」を選択してください。

「カスタムパラメータ」はLTIリンクを作成するときに指定するので、ここでは空欄のままにしてください。

次に「プライバシー」セクションの入力を行います。下図の青枠で示した箇所への入力が必要となります。

![Moodle-ツール設定画面2](images/moodle-411-04.png)

入力が必要となる項目名を以下に示します。

* ランチャのメールをツールと共有する

「ランチャのメールをツールと共有する」は「常に」を選択してください。

最後に「変更を保存する」を選択することでツールの登録が完了します。下図に示すようにツールの管理画面に登録したツールが表示されます。

![Moodle-ツール管理画面2](images/moodle-411-05.png)

### ツールの設定詳細を確認する

CoursewareHubにLTI認証連携の設定を行うために必要となる情報を確認します。

前節の最後に示した「ツールを管理する」画面の「ツール」セクションに、登録したツールの一覧が表示されています。登録したツールの肩に「設定詳細を表示する」リンクがあります（図中に赤丸で示している箇所）。このリンクを選択すると、下図のような画面表示となります。

![Moodle-ツール設定詳細](images/moodle-411-06.png)

表示された情報は、次章で CousewareHub にLTI認証連携の設定を行う際に必要となります。

## CoursewareHubにLTI認証連携設定を行う

### パラメータの指定

Moodleの「ツール設定詳細」に表示された情報のうち以下に示す項目を指定します。

* プラットフォームID
* クライアントID
* デプロイメントID

プラットフォームIDの値を次のセルに入力してください。

> プラットフォームIDに指定する値は、既に登録してあるものと異なる値を指定してください。同じプラットフォームIDを指定した場合、クライアントID、デプロイメントIDの値が異なる値であったとしても後から登録した値のみが有効となります。

In [ ]:
# (例)
# platform_id = 'https://moodle.example.org'

platform_id = 

クライアントIDの値を次のセルに入力してください。

In [ ]:
# (例)
# client_id = 'xxxxxxxxxxxxxxx'

client_id = 

デプロイメントIDの値を次のセルに入力してください。

In [ ]:
# (例)
# deployment_id = '1'

deployment_id = 

### パラメータの保存

前節で指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)
lti_config = gvars.get('lti_config', {})
lti_config[platform_id] = {
    'platform_id': platform_id,
    'client_id': client_id,
    'auth_login_url': f'{platform_id}/mod/lti/auth.php',
    'auth_token_url': f'{platform_id}/mod/lti/token.php',
    'key_set_url': f'{platform_id}/mod/lti/certs.php',
    'deployment_id': deployment_id,
}
gvars['lti_config'] = lti_config
with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

### CoursewareHubにLTI認証連携のための設定ファイルを配置する

前節入力された情報をもとにLTI認証連携のための設定ファイル `lti.json` を生成し、CoursewareHub環境に配置します。

LTI関連の設定ファイルを配置、生成するための Ansible の playbook を実行します。ここで実行するplaybookでは以下の処理を行います。

* 指定されたパラメータに対応するLTIファイルを配置する
* RSAの秘密鍵ファイル`private.key`が存在していない場合は、鍵ペアの生成を行う

まず、実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -CDv -l {target_hub} playbooks/setup-lti.yml || true

実際に設定ファイルの配置を行います。

In [ ]:
!ansible-playbook -Dv -l {target_hub} playbooks/setup-lti.yml

配置した設定ファイル`lti.json`の内容を確認します。

In [ ]:
!ansible {target_hub} -a 'cat {{{{jupyterhub_cfg_dir}}}}/lti/lti.json'

鍵ペアの公開鍵の内容を確認します。

> この公開鍵は将来のLMS連携機能のために使用する。認証連携には使用しない。

In [ ]:
!ansible {target_hub} -b -m shell -a \
    'openssl rsa -pubout -in {{{{jupyterhub_cfg_dir}}}}/lti/private.key 2>/dev/null'

### auth-proxyコンテナにLTI認証連携設定を反映する

前節で更新したLTIの設定ファイル`lti.json`をauth-proxyコンテナに反映します。

auth-proxyコンテナが`lti.json`を読み込み直すためにauth-proxyサービスの再起動を行います。

In [ ]:
!ansible {target_hub} -a 'docker service update --force coursewarehub_auth-proxy'

コンテナの実行状況を確認します。`auth-proxy`コンテナの状態が`Running`であり、かつ新たに起動されていることを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service ps coursewarehub_auth-proxy'

## MoodleでLTI リンクを作成する

MoodleのサイトまたはコースのページにLTIリンクを作成します。

### CoursewareHubへのリンクを作成する



コースの編集モードを開始します。下図の赤丸で示したリンクを選択してください。

> ここではコースにLTIリンクを作成する手順を示しますが、サイトにリンクを作成する場合も同様の手順となります。

![コース画面1](images/moodle-411-10.png)

編集モードの表示画面から「活動またはリソースを追加する」のリンクを選択してください。下図の赤丸で示した箇所などに表示されています。

![コース画面2](images/moodle-411-11.png)

追加する活動またはリソースの選択画面から「外部ツール」を選択してください。

![コース画面3](images/moodle-411-12.png)

下図のような設定画面が表示されます。「活動名」にリンクの名前を入力してください。ここで指定した値が作成されるリンクの表示名前となります。また「事前設定ツール」の項目には登録した外部ツールの名前を選択してください。

![コース画面4](images/moodle-411-13.png)

画面下部の「保存してコースに戻る」ボタンを選択することでLTIリンクが作成されます。

> 「保存して表示」ボタンを選択した場合 iframe の表示となりますが CoursewareHub は埋め込みに対応していな
いため正常に認証・認可されません。

登録したリンクは下図のように表示されます。

![コース画面5](images/moodle-411-14.png)

### CoursewareHubのNotebookへのリンクを作成する

CousewareHubのNotebookファイルへのリンクを作成する場合は「新しい外部ツールを追加する」画面で「カスタムパラメータ」の指定を行います。

前節に示した手順のうち、追加する活動またはリソースの選択画面から「外部ツール」を選択するところまでは同様のの手順となります。「新しい外部ツールを追加する」画面にて「活動名」、「事前設定ツール」を入力した後に、下図の赤丸で示した「さらに表示する」のリンクを選択してください。

![コース画面6](images/moodle-411-15.png)

下図のように「カスタムパラメータ」などの入力欄が表示されます。

![コース画面7](images/moodle-411-16.png)

「カスタムパラメータ」に以下のような内容を指定します。

```
notebook={リンクするNotebookのパス}
```

例えば、CoursewareHubの `textbook/001-test.ipynb` に対するリンクを作成するには、

```
notebook=textbook/001-test.ipynb
```

のような指定を行います。指定するパスはCoursewareHubの各ユーザーのホームディレクトリからの相対パスとなります。それぞれのユーザーは異なるホームディレクトリを持つため、全てのユーザーの同じパスにNotebookのコピーが存在している必要があります。Notebookが見つからない場合は404 のエラー画面が表示されます。